In [19]:
import polars as pl
import numpy as np
import altair as alt
alt.data_transformers.enable("vegafusion")
WINDOW_SIZE = 32

In [143]:
df_train = pl.read_parquet('../data/processed/train_cg_32_10000.parquet')
df_test = pl.read_parquet('../data/processed/test_cg_32_10000.parquet')
len(df_train)/(len(df_test)+len(df_train))

0.7999992948141218

In [83]:
df_train['label'].mean(), df_test['label'].mean()

(0.5023785352982572, 0.5023785248151097)

In [84]:
df_train.head()

read_name,cg_pos,seq,fi,fp,ri,rp,label
str,i64,str,list[i64],list[i64],list[i64],list[i64],i32
"""m64168_200823_191315/1837517/c…",10282,"""AGATCTATTACACAACGTGGTGACCATAGC…","[52, 29, … 21]","[19, 10, … 14]","[62, 14, … 12]","[29, 25, … 13]",1
"""m64168_200820_000733/263719/cc…",5955,"""GAAGGGGCTGATGCCCGGCCTCAGAGGTTA…","[19, 60, … 39]","[12, 49, … 24]","[31, 52, … 28]","[12, 17, … 54]",0
"""m64168_200823_191315/263868/cc…",2321,"""GACGGGGCAGCTGGCCGGGCGGGGGGGCTG…","[23, 59, … 69]","[12, 25, … 9]","[35, 48, … 68]","[18, 11, … 28]",1
"""m64168_200823_191315/1442515/c…",7697,"""TGGAATGCAATGGAACGGAATGGAGTGGGA…","[16, 11, … 21]","[16, 33, … 33]","[22, 18, … 12]","[37, 12, … 11]",1
"""m64168_200823_191315/1638483/c…",7803,"""GATGTACTCCACTTTCGAGCCTGATTCAGA…","[21, 21, … 22]","[44, 21, … 13]","[17, 17, … 21]","[11, 26, … 41]",1


In [85]:
df_train['window_fi'].to_numpy()

ColumnNotFoundError: "window_fi" not found

In [108]:
kinetic_cols = ['fi', 'fp', 'ri', 'rp']
data_array = np.stack([df_train.slice(0,1)[c].to_numpy() for c in kinetic_cols], axis=1)
data_array.shape

(1, 4)

In [125]:
lengths = df_train["fi"].list.len()
all_equal = lengths.n_unique() == 1
print(all_equal)

True


In [153]:
df_train = pl.read_parquet('../data/processed/train_cg_32_10000.parquet')
df_train = df_train.with_columns([
    # pl.col("seq").list.to_array(WINDOW_SIZE),  
    pl.col("fi").list.to_array(WINDOW_SIZE),  
    pl.col("fp").list.to_array(WINDOW_SIZE), 
    pl.col("ri").list.to_array(WINDOW_SIZE), 
    pl.col("rp").list.to_array(WINDOW_SIZE), 
    ])


In [ ]:

arr = np.stack([df_train[col].to_numpy() for col in kinetic_cols], axis=1)
arr.shape

(1361342, 4, 32)

In [156]:
unique_chars = (
    df_train.select(
        pl.col("seq").str.explode().unique().alias("unique_chars")
    )
)
unique_chars

/var/folders/wt/33m104j54lb3xnbgck84qsw40000gp/T/ipykernel_1306/1930145770.py:3: DeprecationWarning: `str.explode` is deprecated; use `str.split("").explode()` instead. Note that empty strings will result in null instead of being preserved. To get the exact same behavior, split first and then use a `pl.when...then...otherwise` expression to handle the empty list before exploding.
  pl.col("seq").str.explode().unique().alias("unique_chars")


unique_chars
str
"""C"""
"""T"""
"""A"""
"""G"""


In [162]:
vocab = {'A':0, 'T':1, 'C':2, 'G':3}
seq_ints = (
    df_train["seq"]
    .str.split("")                # Split into characters
    .list.eval(                   # Evaluate an expression on each list element
        pl.element().replace_strict(vocab)
    )
    .to_numpy()                   # Convert to NumPy array
)
np.stack(seq_ints).shape

(1361342, 32)

In [92]:
np.stack([df_train[:5][c].to_numpy() for c in kinetic_cols], axis=1)

array([[array([52, 29, 39, 35, 20, 23,  6, 23, 34, 40, 38, 35, 18, 69, 22, 35, 21,
               22, 18, 15, 29, 22, 27, 66, 62, 45, 23, 37, 11,  9,  8, 21])       ,
        array([19, 10, 31, 16, 26, 12, 13, 23, 21,  9, 19,  8, 26, 17, 16, 22, 19,
               12, 38, 27, 19, 30, 41, 13, 14, 12, 12, 12, 17, 38, 13, 14])       ,
        array([ 62,  14,  26,  15,  71,  30,  30,  19,  12,  18,  24,  23,  36,
               139, 176,  49,  50,  49,  26,  34,  33,  27,  23,  26,  37,  38,
                32,  22,  59,  28,  43,  12])                                  ,
        array([29, 25, 19, 26, 16, 16,  7, 32, 39, 12, 33, 11, 10, 12, 28,  7, 14,
               16, 20, 24, 34, 16, 11, 15, 10, 14, 28, 22, 27, 24, 62, 13])       ],
       [array([ 19,  60,  29,  50,  45,  29,  35,  14,  14,  59,  39,  20,  81,
                56,  33,  52,  21,  22,  50,  23, 126,  70,  63,  25, 145,  43,
                55,  20,  48,  26,  24,  39])                                  ,
        array([1

In [109]:
list_of_2d_arrays: np.stack([np.array(df_train[c].to_list()) for c in kinetic_cols], axis=1)

In [111]:
list_of_2d_arrays

[array([[52, 29, 39, ...,  9,  8, 21],
        [19, 60, 29, ..., 26, 24, 39],
        [23, 59, 14, ..., 24, 70, 69],
        ...,
        [11, 28, 12, ..., 19, 67, 16],
        [16, 25, 22, ..., 14, 12, 14],
        [26, 91, 17, ..., 57, 21, 50]], shape=(1361342, 32)),
 array([[19, 10, 31, ..., 38, 13, 14],
        [12, 49, 15, ...,  9, 11, 24],
        [12, 25, 16, ..., 35, 12,  9],
        ...,
        [12, 17, 21, ..., 32, 24, 28],
        [14, 16, 52, ..., 29, 12, 58],
        [22, 39, 17, ..., 25,  7, 22]], shape=(1361342, 32)),
 array([[62, 14, 26, ..., 28, 43, 12],
        [31, 52, 26, ..., 34, 20, 28],
        [35, 48,  5, ..., 31, 65, 68],
        ...,
        [16, 17, 16, ..., 31, 10, 24],
        [17, 13,  6, ..., 17,  9, 19],
        [ 9, 11, 15, ..., 13, 25,  5]], shape=(1361342, 32)),
 array([[29, 25, 19, ..., 24, 62, 13],
        [12, 17, 20, ..., 24, 28, 54],
        [18, 11, 11, ..., 19,  7, 28],
        ...,
        [12, 18, 28, ..., 27, 23, 29],
        [19, 39, 45, 

In [3]:
df_train = pl.read_parquet('../data/processed/test_run_train.parquet')
df_test = pl.read_parquet('../data/processed/test_run_test.parquet')
len(df_train)/(len(df_test)+len(df_train))

0.7999774431850223

In [7]:
df_test['label'].mean()

0.4990132506343389

In [17]:
import re
seq = 'ATCGATCGAA'
cgs = re.finditer('CG', seq)
print(seq)
for match in cgs:
    print(match.start())

ATCGATCGAA
2
6


In [27]:
df_train_singletons = pl.read_parquet('../data/processed/1k_singletons_train.parquet')
df_test_singletons = pl.read_parquet('../data/processed/1k_singletons_test.parquet')
len(df_train_singletons)


85155